### 0. Installing Packages

### 1. Importing Neccessary Packages

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')
import json
import re
import wandb
import os
import unicodedata
from pprint import pprint
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login, login
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from huggingface_hub import notebook_login



In [2]:
notebook_login()

In [3]:
import wandb

# Start a W&B run
wandb.init(project="news-summarization2", entity="lyutovad")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lyutovad. Use `wandb login --relogin` to force relogin


### Mistral Instruct v0.2:

##### Key Characteristics:

- __Model Size__: 7 миллиардов параметров
- __Architecture__: трансформер, аналогичная GPT-3
- __Fine-tuning__: Специально настроена для выполнения инструкций и сохранения темы
- __Training Data__: Включает массивный набор данных Pile и, вероятно, другие текстовые источники
-  __Instruction Format__: Использует токены [INST] и [/INST] для разграничения инструкций
- __Context Window__: Увеличено до 32k токенов в версии v0.2 (vs. 8k в v0.1) 
- __Rope-theta__: Установлено значение 1e6 для большего акцента на понимание языка
- __Sliding-Window Attention__: Удалено в версии 0.2 для потенциального повышения эффективности

In [4]:
DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

In [5]:
df = pd.read_csv("text_sum.csv")
df.columns = [str(q).strip() for q in df.columns]

dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['text', 'summary'],
    num_rows: 26921
})

#### Prompt

In the following function we'll be merging our `article` and `summary` columns by creating the following template:

```
<s>### Instruction:
Summarize the following text in 2-4 sentences. Focus on countries, products and numbers. If goods can be grouped in one group with the same name, it is necessary to do so. Don't enclude names and introductory words. be impersonal and use bullet points.

### Input:
{article}

### Response:
{summary}</s>
```

In [6]:
DEFAULT_SYSTEM_PROMPT = "Summarize the following text in 2-4 sentences. Focus on countries, products and numbers. If goods can be grouped in one group with the same name, it is necessary to do so. Don't enclude names and introductory words. be impersonal and use bullet points."
def generate_training_prompt(
    text: str, summary: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    bos_token = "<s>"
    eos_token = "</s>"
    
    full_prompt = ""
    full_prompt += bos_token
    full_prompt += "### Instruction:"
    full_prompt += "\n" + system_prompt
    full_prompt += "\n\n### Input:"
    full_prompt += "\n" + text
    full_prompt += "\n\n### Response:"
    full_prompt += "\n" + summary
    full_prompt += eos_token
    return full_prompt


In [7]:
def create_prompt(data_point):
    text, summary = data_point["text"], data_point["summary"]
    text = unicodedata.normalize("NFKD", text)
    text = re.sub(r'\s(\.)\s', ' ', text)
    
    summary = re.sub(r'([^a-zA-Z\s])?\n(\w+)', r'\1 \2', summary)
    summary = re.sub(r'\s(\.)', '.', summary)
    return text.strip(), summary.strip()


def generate_text(data_point):
    text, summary = create_prompt(data_point)
    return generate_training_prompt(text, summary)


# Example usage with a new dataset format
example_data_point = {
    "id": "train_0",
    "text": df.iloc[5]["text"],
    "summary": df.iloc[5]["summary"]
}


example = generate_text(example_data_point)
print(example)

<s>### Instruction:
Summarize the following text in 2-4 sentences. Focus on countries, products and numbers. If goods can be grouped in one group with the same name, it is necessary to do so. Don't enclude names and introductory words. be impersonal and use bullet points.

### Input:
Шанхай, 22 ноября /Синьхуа/ -- В январе-октябре текущего года общий объем внешней торговли Шанхая /Восточный Китай/ вырос на 5,3 проц. в годовом исчислении до 3,46 трлн юаней /около 483 млрд долл. США/ в стоимостном выражении. Об этом свидетельствуют опубликованные во вторник данные Шанхайской таможни. В частности, за отчетный период объемы экспорта и импорта Шанхая выросли на 12,5 и 0,9 проц. до 1,41 трлн юаней и 2,05 трлн юаней соответственно по сравнению с тем же периодом прошлого года. Объем внешнеторгового оборота этого мегаполиса за январь-октябрь с.г. составил около 10 проц. от общего объема внешней торговли страны, сообщила Шанхайская таможня. Согласно данным, на долю предприятий с ин

In [8]:
# Split the processed dataset into train, validation, and test sets
train_dataset = dataset.shuffle(seed=21).select(range(0, int(0.8 * len(dataset))))
validation_dataset = dataset.shuffle(seed=21).select(range(int(0.8 * len(dataset)), int(0.9 * len(dataset))))
test_dataset = dataset.shuffle(seed=21).select(range(int(0.9 * len(dataset)), len(dataset)))

### Загрузка модели

Загружается модель `4bit` с двойным квантованием и `bfloat16` в качестве dtype.

В данном случае мы используем модель, настроенную по инструкции, вместо базовой модели. Для тонкой настройки базовой модели потребуется гораздо больше данных!

In [9]:
def create_model_and_tokenizer():
    nf4_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_quant_type = "nf4",
        bnb_4bit_use_double_quant = True,
        bnb_4bit_compute_dtype = torch.bfloat16
    )


    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map='auto',
        quantization_config=nf4_config,
        use_cache=False
    )
    
    model.config.use_cache = False
    
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

In [11]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

model, tokenizer = create_model_and_tokenizer()
# model.config.use_cache = False

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [12]:
# if torch.cuda.device_count() > 1: # If more than 1 GPU
#     model.is_parallelizable = True
#     model.model_parallel = True
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 'load_in_8bit': False,
 'load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': True,
 'bnb_4bit_compute_dtype': 'bfloat16'}

In [13]:
def generate_response(prompt, model):
    encoded_input = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encoded_input.to('cuda')
    
    generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    
    decoded_output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    
    return decoded_output[0].replace(prompt,"")

In [14]:
prompt="""### Instruction:\nSummarize the following text in 2-4 sentences. Focus on countries, products and numbers. If goods can be grouped in one group with the same name, it is necessary to do so. Don't enclude names and introductory words. be impersonal and use bullet points.\n\n### Response:"""

In [15]:
generate_response(prompt, model)

'\n\n- The text discusses international trade volume between six countries: Australia, China, Germany, Japan, United Kingdom, and United States.\n- China exported the most goods, exceeding $2.4 trillion, followed by the United States with $2.1 trillion. Germany had the third-highest export volume, amounting to approximately $1.4 trillion.\n- Australia had the lowest export volume among the six, with around $111'

In [16]:
# Set LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

In [17]:
from peft import *

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [19]:
def generate_inference_prompt(
    text: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    full_prompt = ""
    full_prompt += "### Instruction:"
    full_prompt += "\n" + system_prompt
    full_prompt += "\n\n### Input:"
    full_prompt += "\n" + text
    full_prompt += "\n\n### Response:"
    
    return full_prompt

def generate_response(prompt, model):
    encoded_input = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    model_inputs = encoded_input.to('cuda')
    
    generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    
    decoded_output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    
    return decoded_output[0].replace(prompt,"")


def generate_summaries(model, dataset, num_samples=5):
    summaries = []
    for i, example in enumerate(dataset):
        if i >= num_samples:
            break
        print(i)
        prompt = generate_inference_prompt(example['text'])
        summary = generate_response(prompt, model)
        summaries.append({'text': example['text'], 'generated_summary': summary, 'original_summary': example})
    return summaries

In [20]:
# Generate summaries before fine-tuning
original_summaries = generate_summaries(model, test_dataset, num_samples=5)

# Convert to DataFrame and log to W&B
df_original = pd.DataFrame(original_summaries)
wandb.log({"original_summaries": wandb.Table(dataframe=df_original)})

0
1
2
3
4


In [22]:
##############################
# TrainingArguments parameters
##############################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "news-summarization-mistral-7b-finetuned"

# Number of training epochs
# num_train_epochs = 5

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

In [28]:
training_arguments = TrainingArguments(
    per_device_train_batch_size = 4,
    logging_steps = 10,
    learning_rate = 2e-4,
    max_steps = 100, # the total number of training steps to perform
    num_train_epochs = 3,
    evaluation_strategy="epoch",
    warmup_steps = 0.03,
    save_strategy="epoch",
    group_by_length = True,
    output_dir = output_dir,
    report_to="wandb",  
    save_safetensors=True,
    lr_scheduler_type = 'constant',
    load_best_model_at_end = True,
    push_to_hub = True,
    push_to_hub_model_id = "news-summarization-finetuned-mistral-7b",
)

In [30]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    peft_config=peft_config,
    formatting_func=generate_text,
    max_seq_length=300,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=True
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [31]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

with torch.no_grad():
    torch.cuda.empty_cache()

# Fine-tune your model
trainer.train()


# Generate summaries after fine-tuning
fine_tuned_summaries = generate_summaries(trainer.model, test_dataset, num_samples=5)


# Convert to DataFrame and log to W&B
df_fine_tuned = pd.DataFrame(fine_tuned_summaries)
wandb.log({"fine_tuned_summaries": wandb.Table(dataframe=df_fine_tuned)})

Epoch,Training Loss,Validation Loss
0,1.332900,1.409050


0
1
2
3
4


In [32]:
# Generate summaries after fine-tuning
fine_tuned_summaries = generate_summaries(trainer.model, test_dataset, num_samples=5)


# Convert to DataFrame and log to W&B
df_fine_tuned = pd.DataFrame(fine_tuned_summaries)
wandb.log({"fine_tuned_summaries": wandb.Table(dataframe=df_fine_tuned)})

0
1
2
3
4


In [33]:
df_fine_tuned["generated_summary"][4]

'\nЗа рубеж вышла информация о том, что Россия стала крупнейшим поставщиком сельскохозяйственной продукции, прежде всего овощей и картофеля, в Турцию. Американские поставки снизились на 6% в августе года по сравнению с тем же ассортиментом а прежнем году. Однако поставки в Турцию из США продолжают оставаться высокими и превысили 125,3 тыс. тонн в августе. Поставки из России выросли на 50% по сравнению с августом 2021 года, достигнув 75,8 тыс. тонн. Поставки продукции состави'

In [34]:
prompt="""### Instruction:\nSummarize the following text in 2-4 sentences. Focus on countries, products and numbers. If goods can be grouped in one group with the same name, it is necessary to do so. Don't enclude names and introductory words. be impersonal and use bullet points.\n\n### Input:\nРоссия и Китай намерены продолжать углубление взаимовыгодного сотрудничества в различных форматах на фоне нестабильности мировой экономики и западных санкций. Об этом говорится в совместном заявлении обеих стран по итогам девятого китайско-российского финансового диалога в Пекине, которое было принято 18 декабря. «Стороны будут углублять сотрудничество в рамках многосторонних форматов, включая G20, БРИКС и АТЭС, с целью ограничения рисков, связанных с геополитической и геоэкономической фрагментацией, последовательно продвигать экономическую глобализацию и поддерживать стабильность и бесперебойность глобальных промышленных цепочек и цепочек поставок», — цитирует данное заявление «РИА Новости». Отмечается, что развитие мировой экономики пока что остается неопределенным в связи с продолжительной и высокой инфляцией, последствиями пандемии коронавируса, геополитической напряженностью и политически мотивированными ограничениями. В связи с этим стороны договорились придерживаться в рамках сотрудничества принципов взаимной выгоды, а также подтвердили приверженность усилению экономических связей с помощью координации макроэкономической политики. Кроме того, РФ и КНР выразили поддержку расширения БРИКС, которое названо отправной точкой для наращивания усилий по финансово-экономическому сотрудничеству стран объединения. Ранее в этот день вице-премьер РФ Дмитрий Чернышенко заявил после заседания российско-китайской межправкомиссии по подготовке регулярных встреч глав правительств, что Россия и Китай планируют к 2030 году увеличить объем товарооборота до $300 млрд. По его словам, за 11 месяцев 2023 года торговый оборот обеих стран уже превысил $201 млрд. До этого, 15 декабря, полпред президента РФ в Дальневосточном федеральном округе (ДФО) Юрий Трутнев сообщил, что за 10 лет товарооборот между дальневосточными регионами России и северо-восточными провинциями Китая увеличился вдвое. По его данным, в 2022 году данный показатель составил $22 млрд. Ожидается, что в 2023-м он достигнет $27 млрд. В тот же день вице-премьер РФ Александр Новак сообщил, что Россия и Китай договорились о сотрудничестве в области технологий по производству водорода. В свою очередь, премьер Госсовета КНР Дин Сюэсян после встречи с Новаком также отметил, что энергетическое сотрудничество двух стран в текущем году активно развивалось и достигло новых результатов даже в условиях сложной и напряженной международной обстановки. 7 декабря президент России Владимир Путин заявил на пленарном заседании форума «Россия зовет!», что Москва готова к всестороннему сотрудничеству с Пекином. Он подчеркнул, что РФ и КНР думают о перспективах и будущем мироустройстве, а также отходят от модели «купил-продал» в двусторонних отношениях.\n\n### Response:"""
generate_response(prompt, model)

'\nРоссия и Китай намерены углублять экономическое сотрудничество. Об этом говорится в совместном заявлении РФ и КНР по итогам девятого китайско-российского финансового диалога. Около $201 млрд Объём товарооборота между РФ и КНР по итогу 11 месяцев 2023 года составил $201 млрд, показатель за 2022 год составил $22 млрд. К 2030 году между сторонами планируется возможность увеличить объем товарооборота до $300 млрд. Двустороннее сотрудничество разви'

In [80]:
# trainer.push_to_hub()

In [81]:
# trainer.save_model("news-summarization-finetuned-mistral-7b-instruct-v0.2")